In [ ]:
# imports
import warnings
warnings. filterwarnings('ignore')

import numpy
from qpsolvers import solve_qp
from  qiskit.chemistry import FermionicOperator
from qiskit.aqua.operators.legacy.op_converter import to_weighted_pauli_operator
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit import Aer
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.operators.legacy import op_converter
from qiskit.aqua.algorithms import VQE
from qiskit.aqua import QuantumInstance
from tqdm import tqdm
from joblib import Parallel, delayed
import itertools
from qiskit import QuantumRegister, QuantumCircuit, execute, ClassicalRegister
from qiskit.circuit.library import U3Gate
from qiskit.aqua.components.initial_states import Custom
from qiskit.chemistry.components.initial_states import HartreeFock
import scipy
import matplotlib.pyplot as plt
from qiskit.quantum_info import partial_trace, Statevector

In [ ]:
# 2 site Hubbard model parameters
t = 1 #hopping factor
U = 2 #coulomb repulsion factor
mu = U/2 #chemical potential factor

In [ ]:
# 2x1 Hubbard Hamiltonian
def HubbardHamiltonian(U,t,num_spin_orbitals,num_particles):
    h1=numpy.zeros((4,4))
    h2=numpy.zeros((4,4,4,4))
    num_sites=int(num_spin_orbitals // 2)
    for i in range(num_sites - 1):
        h1[i, i + 1] = h1[i + 1, i] = -t
        h1[i + num_sites, i + 1 + num_sites] = h1[i + 1 + num_sites, i + num_sites] = -t
        h1[i][i] = -mu
        h1[i + num_sites][i + num_sites] = -mu
    h1[num_sites - 1][num_sites - 1] = -mu
    h1[2 * num_sites - 1][2 * num_sites - 1] = -mu    
    h1[0, num_sites - 1] = h1[num_sites - 1, 0] = -t
    h1[num_sites, 2 * num_sites - 1] = h1[2 * num_sites - 1, num_sites] = -t
    for i in range(num_sites):
        h2[i, i , i + num_sites, i + num_sites] = U
    fermion_op = FermionicOperator(h1 = h1, h2 = h2) # Fermionic Hamiltonian
    qubit_op = fermion_op.mapping('jordan_wigner') #Qubit Hamiltonian
    return qubit_op

In [ ]:
# construct the qubit operator rep. of the 2x1 Hubbard model and then the matrix representation
qubit_H = HubbardHamiltonian(U = U, t = 1, num_spin_orbitals = 4, num_particles = 2)
#constructing matrix rep. in the Fock space
H_mat=op_converter.to_matrix_operator(qubit_H).dense_matrix

In [ ]:
# compute exact ground state energy and wavefunction through diagonalization
w,v = numpy.linalg.eigh(H_mat)
Eg = w[0]
# print("ground state energy-", w[0])
state_g = v[:,0]
# print("ground state wvfn.", state_g)

In [ ]:
def rotated_state(labels,params,state0):
    U=WeightedPauliOperator([[1,Pauli.from_label('IIII')]])
    for i in range(len(labels)):
        U=WeightedPauliOperator([[numpy.cos(params[i]),Pauli.from_label('IIII')],[-1j*numpy.sin(params[i]),Pauli.from_label(labels[i])]]).multiply(U)
        U_mat=op_converter.to_matrix_operator(U).dense_matrix
        rot_state=numpy.dot(U_mat,state0) 
    return rot_state

In [ ]:
def TimeEvolutionOperator(T):
    return numpy.dot(numpy.dot(v,numpy.diag(numpy.exp(-1j*w*T))),numpy.conjugate(v.T))

### $G_{1,2}^{\uparrow,\uparrow}(t>0)=\langle G|e^{iHT}c_{1\uparrow}(0)e^{-iHT}c^{\dagger}_{2\uparrow}(0)|G\rangle$, $c^{\dagger}_{2\uparrow}(0)=IIXZ+iIIYZ$, <br>
### $|\mathcal{E}\rangle = IIXZ|G\rangle= e^{i\frac{\pi}{2}IIXZ}e^{i\frac{2\pi}{27}IZXY}e^{i\frac{\pi}{4}XYII}e^{i\frac{\pi}{4}IIXY}e^{-i\frac{\pi}{2}}|G\rangle$, <br>

### also constructing $IIIY|G\rangle$ 

In [ ]:
# excited state 1 and 2

exc_labels = ['IIII','IIXZ']
exc_params = numpy.array([-numpy.pi/2,numpy.pi/2])
exc_state = rotated_state(exc_labels,exc_params,state_g)

exc_labels2 = ['IIII','IIIY']
exc_params2 = [-numpy.pi/2,numpy.pi/2]
exc_state2 = rotated_state(exc_labels2,exc_params2,state_g)
exc_state2[numpy.abs(exc_state)<1e-5] = 0

In [ ]:
# greens function evolution
def greens_function(T, dT, T0):

    steps = int((T-T0)/dT)
    T_arr = numpy.linspace(T0, T, steps)
    GF_exact = []
    for i in tqdm(range(len(T_arr))):
        U_T = TimeEvolutionOperator(T_arr[i])
        exact_evolved_state = numpy.dot(U_T, exc_state)
        G1 = numpy.exp(1j*(U-rho)/2.)*numpy.dot(numpy.conjugate(exc_state2), exact_evolved_state)
        GF_exact.append(G1)
    return GF_exact

In [ ]:
# parameters for greens function
T = 30
dT = 0.1
T0 = 0
steps = int((T-T0)/dT)
T_arr = numpy.linspace(T0,T,steps)
rho = numpy.sqrt(U**2+16*t*t)
G = greens_function(T,dT,T0)

In [ ]:
# graphing greens function and spectral function
# fig, ax = plt.subplots(1,2)
# plt.rcParams["figure.figsize"] = (40, 20)

# ax[0].tick_params(labelsize=30)
# ax[0].plot(T_arr, numpy.real(G), color='black')

# """SPECTRAL FUNCTION"""
# # Number of sample points
# # num_samp1=len(G)
# # sample spacing
# # ImgGf = numpy.fft.fft(numpy.imag(G))
# # Tf1 = numpy.linspace(0, 40, num_samp1//2)
# # ax[1].set_yscale('log')
# # ax[1].tick_params(labelsize=20)
# # ax[1].plot(Tf1, 2.0/num_samp1 * numpy.abs(ImgGf[:num_samp1//2])/numpy.pi, color='black', linestyle='-')
# # ax[1].plot(-Tf1, 2.0/num_samp1 * numpy.abs(ImgGf[:num_samp1//2])/numpy.pi, color='black', linestyle='-')
# # ax[1].plot(Tf1, 2.0/num_samp1 * numpy.abs(ImgGf[:num_samp1//2])/numpy.pi, color='black', linestyle='-')
# # ax[1].plot(-Tf1, 2.0/num_samp1 * numpy.abs(ImgGf[:num_samp1//2])/numpy.pi, color='black', linestyle='-')
# # ax[1].plot(T_arr,numpy.imag(G),linestyle='-')

# plt.show()

In [ ]:
# generators and angles for constructing adaptive ansatz for the 2x1 Hubbard model at U=2 t=1
labels=['IIXY', 'XYII', 'IZXY']

# U = 2
params = [-0.7853980948120887, -0.7853983093282092, 0.23182381954801887]

# U = 3
# params = [0.7853959259806095,  0.7853996767775284, -1.2490064682759752]

In [ ]:
#circuit initialization
init_circ = QuantumCircuit(2*2)
init_circ.x(0)
init_circ.x(2)
init_state_circuit=Custom(4, circuit = init_circ)
init_state = init_state_circuit #HartreeFock(num_spin_orbitals,num_particles=4,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
var_form_base = UCCSD(4,num_particles=2, initial_state=init_state,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
backend = Aer.get_backend('statevector_simulator')
optimizer = L_BFGS_B()

#adaptive circuit construction
var_form_base.manage_hopping_operators()
circ0 = var_form_base.construct_circuit(parameters = [])
state0 = execute(circ0,backend).result().get_statevector()
state0[numpy.abs(state0)<1e-5] = 0
adapt_state = rotated_state(labels, params, state0)

In [ ]:
# checking inner product between numerical and exact ground state
print("overlap between analytic and numerical ground state is-",numpy.dot(state_g,adapt_state))

In [ ]:
# confirming exact energy

# check expectation value of the Hamiltonian with respect to adaptive ansatz
def expectation_op(Op,state):
    return numpy.dot(numpy.dot(state,Op),numpy.conjugate(state))

E_adapt = expectation_op(H_mat,adapt_state)

# print("exact energy-",Eg)
# print("Energy from adaptive ansatz-",E_adapt)
# print("convergence error", E_adapt-Eg)

In [ ]:
# including an excitation to the adaptive ansatz ground state

In [ ]:
# constructing the excited state ansatz
exc_labels = ['IIII','IIXZ']
exc_params = numpy.array([-numpy.pi/2,numpy.pi/2])
exc_state = rotated_state(exc_labels,exc_params,adapt_state)
exc_state[numpy.abs(exc_state)<1e-5] = 0

In [ ]:
# exact excited state
exact_exc_state=rotated_state(exc_labels,exc_params,state_g)
#checking inner product between numerical and analytic state
print("overlap between analytic and numerical exc. state is-",numpy.dot(numpy.conjugate(exact_exc_state),exc_state))

In [ ]:
def M(p,q,vqs_generators,vqs_params,ref_state):
    thetas=numpy.array(vqs_params)
    shift_1=numpy.array([0]*(p)+[numpy.pi/2]+[0]*(len(vqs_params)-p-1))
    shift_2=numpy.array([0]*(q)+[numpy.pi/2]+[0]*(len(vqs_params)-q-1))
    state_1=rotated_state(vqs_generators,vqs_params+shift_1,ref_state)
    state_2=rotated_state(vqs_generators,vqs_params+shift_2,ref_state)
    M_arr=numpy.real(numpy.dot(numpy.conjugate(state_1),state_2))
    return M_arr

In [ ]:
def V(p,vqs_generators,vqs_params,ref_state):
    thetas=numpy.array(vqs_params)
    shift_1=numpy.array([0]*(p)+[numpy.pi/2]+[0]*(len(vqs_params)-p-1))
    state_1=rotated_state(vqs_generators,vqs_params+shift_1,ref_state)
    state=rotated_state(vqs_generators,vqs_params,ref_state)
    V_arr=numpy.imag(numpy.dot(numpy.dot(numpy.conjugate(state_1),H_mat),state))
    return V_arr

# Alex stuff

In [ ]:
# basic setup
import numpy as np
import copy

PAULI_X = np.array([[0,1],[1,0]], dtype='complex128')
PAULI_Y = np.array([[0,-1j],[1j,0]], dtype='complex128')
PAULI_Z = np.array([[1,0],[0,-1]], dtype='complex128')
IDENTITY = np.eye(2, dtype='complex128')

def pauli_string_to_matrix(pauli_string):
    return Pauli(pauli_string).to_matrix()

def pauli_string_exp_to_matrix(pauli_string, param):
    return expm(-1j * param * Pauli(pauli_string).to_matrix())


backend = Aer.get_backend('statevector_simulator')
qasm_backend = Aer.get_backend('qasm_simulator')

# circuit creation

def rotate_state(pauli_string, param, circuit):
    ancilla_boolean = (1 if circuit.num_qubits == 5 else 0)
    if pauli_string == 'IIII':
        gate = 1
        for j in range(len(pauli_string)):
            gate = np.kron(gate, IDENTITY)
        gate *= -1j * np.sin(param)
        gate += np.cos(param) * np.eye(16)
        qubits_to_act_on = [1,2,3,4] if ancilla_boolean else [0,1,2,3]
        circuit.unitary(gate, qubits_to_act_on, label=pauli_string)
    else:
        qubits_to_act_on = []
        gate = 1
        for j in range(len(pauli_string)):
            if pauli_string[j] == 'X':
                gate = np.kron(gate, PAULI_X)
            elif pauli_string[j] == 'Y':
                gate = np.kron(gate, PAULI_Y)
            elif pauli_string[j] == 'Z':
                gate = np.kron(gate, PAULI_Z)
            if pauli_string[j] != 'I':
                qubits_to_act_on.append(np.abs(j - 3) + (0,1)[ancilla_boolean])
        gate *= (-1j * np.sin(param))
        gate += np.cos(param) * np.eye(2**len(qubits_to_act_on))
        qubits_to_act_on.reverse()
        circuit.unitary(gate, qubits_to_act_on, label = pauli_string)
        circuit.barrier()

def create_initial_state():
    circuit = QuantumCircuit(4)
    circuit.x(0)
    circuit.x(2)
    circuit.barrier()
    return circuit

def create_adapt_ground_state():
    labels = ['IIXY', 'XYII', 'IZXY']
    params = [-0.7853980948120887, -0.7853983093282092, 0.23182381954801887]
    circuit = create_initial_state()
    for i in range(len(labels)):
        rotate_state(labels[i], params[i], circuit)
    return circuit

def create_excited_state():
    labels=['IIXY', 'XYII', 'IZXY', 'IIII', 'IIXZ']
    params=[-0.7853980948120887, -0.7853983093282092, 0.23182381954801887,numpy.pi/2,-numpy.pi/2.] 
    circuit = create_initial_state()
    for i in range(len(labels)):
        rotate_state(labels[i], params[i], circuit)
    circuit.barrier()
    return circuit

def create_excited_state2():
    labels = ['IIXY', 'XYII', 'IZXY', 'IIII', 'IIIY']
    params = [-0.7853980948120887, -0.7853983093282092, 0.23182381954801887, -numpy.pi/2, numpy.pi/2]
    circuit = create_initial_state()
    for i in range(len(labels)):
        rotate_state(labels[i], params[i], circuit)
    return circuit

excited_state = execute(create_excited_state(), backend).result().get_statevector()
excited_state2 = execute(create_excited_state2(), backend).result().get_statevector()

In [ ]:
def create_circuit_ancilla(state, excitations, nonlinear):
    circuit = QuantumCircuit(5)
    circuit.x(1)
    circuit.x(3)
    labels = ['IIXY', 'XYII', 'IZXY']
    params = [-0.7853980948120887, -0.7853983093282092, 0.23182381954801887]
    # this creates|G>
    
    # if state == 2, use first_excitation
    if state == 'exc_state':
        labels.extend(excitations[0])
        params.extend([numpy.pi/2,-numpy.pi/2.])
    
    if nonlinear['bool']:
        labels.extend(nonlinear['generators'])
        params.extend(nonlinear['params'])
        if state == 'exc_state':
            labels.extend(excitations[1])
            params.extend([numpy.pi/2,-numpy.pi/2.])
    
    #applies gates
    for i in range(len(labels)):
        rotate_state(labels[i], params[i], circuit)   

    circuit.barrier()
    return circuit

In [ ]:
def controlled_rotate_state(pauli_string, param, circuit):
    if pauli_string == 'IIII':
        return
    num_qubits = 4 #the ancilla does not count
    qubits_to_act_on = []
    gate = 1
    for j in range(len(pauli_string)):
        if pauli_string[j] == 'X':
            gate = np.kron(gate, PAULI_X)
        elif pauli_string[j] == 'Y':
            gate = np.kron(gate, PAULI_Y)
        elif pauli_string[j] == 'Z':
            gate = np.kron(gate, PAULI_Z)
        if pauli_string[j] != 'I':
            qubits_to_act_on.append(np.abs(j - num_qubits + 1) + 1)
    qubits_to_act_on.reverse()

    #convert unitary to gate through a temporary circuit
    temp_circuit = QuantumCircuit(2)
    temp_circuit.unitary(gate, [0, 1]) #we only have controlled 2-qubit unitaries: IIXX, XXII, IIYY, YYII, ZIZI, IZIZ
    controlled_gate = temp_circuit.to_gate(label = 'Controlled ' + pauli_string).control(1)
    qubits_to_act_on.insert(0, 0) #insert ancilla bit to front of list
    circuit.append(controlled_gate, qubits_to_act_on)

In [ ]:
def measure_ancilla(circuit, shots):
    classical_register = ClassicalRegister(1, 'classical_reg')
    circuit.add_register(classical_register)
    circuit.measure(0, classical_register[0])

    result = execute(circuit, qasm_backend, shots = shots).result() 
    counts = result.get_counts(circuit)
    if counts.get('0') != None:
        return 2 * (result.get_counts(circuit)['0'] / shots) - 1
    else:
        return -1

def measure_ancilla_statevector(circuit):
    full_statevector = Statevector(circuit)
    partial_density_matrix = partial_trace(full_statevector, [1, 2, 3, 4])
    partial_statevector = np.diagonal(partial_density_matrix)
    return ((2 * partial_statevector[0]) - 1).real    

In [ ]:
def calculate_m_statevector(p, q, vqs_generators, vqs_params, state, excitations, nonlinear):
    
    # for the ground state, creates either |G> or e^{iHT}|G>
    # for the excited state, creates either P_1|G> or e^{iHT}P_1|G>
    circuit = create_circuit_ancilla(state, excitations, nonlinear)

    circuit.h(0)
    circuit.x(0)
    circuit.barrier()
    
    for i in range(0, p):
        rotate_state(vqs_generators[i], vqs_params[i], circuit)
    circuit.barrier()

    controlled_rotate_state(vqs_generators[p], vqs_params[p], circuit)
    circuit.barrier()

    for i in range(p, q):
        rotate_state(vqs_generators[i], vqs_params[i], circuit)
    circuit.barrier()

    circuit.x(0)
    controlled_rotate_state(vqs_generators[q], vqs_params[q], circuit)
    circuit.h(0)
    circuit.barrier()
    return measure_ancilla_statevector(circuit)

def calculate_v_statevector(p, vqs_generators, vqs_params, state, excitations, nonlinear):
    n_theta = len(vqs_params)    
    circuit = create_circuit_ancilla(state, excitations, nonlinear)

    circuit.h(0)
    circuit.x(0)
    
    for i in range(0, p):
        rotate_state(vqs_generators[i], vqs_params[i], circuit)
    circuit.barrier()

    controlled_rotate_state(vqs_generators[p], vqs_params[p], circuit)
    circuit.barrier()

    for i in range(p, n_theta):
        rotate_state(vqs_generators[i], vqs_params[i], circuit)
    circuit.barrier()

    circuit.x(0)

    coeffs = [0.5, 0.5, -0.5, -0.5, -0.5, -0.5, -1.0]
    measurements = []
    for i in range(len(coeffs)):
        single_h_circuit = copy.deepcopy(circuit)
        controlled_rotate_state(vqs_generators[i], coeffs[i], single_h_circuit)
        single_h_circuit.h(0)
        measurements.append(measure_ancilla_statevector(single_h_circuit))
    results = 0
    for i in range(len(coeffs)):
        results += measurements[i] * coeffs[i]
    return results

def calculate_m_shots(p, q, vqs_generators, vqs_params, shots, state, nonlinear):
    circuit = create_circuit_ancilla(state, nonlinear) #Creates |E>

    circuit.h(0)
    circuit.x(0)
    circuit.barrier()
    
    for i in range(0, p):
        rotate_state(vqs_generators[i], vqs_params[i], circuit)
    circuit.barrier()

    controlled_rotate_state(vqs_generators[p], vqs_params[p], circuit)
    circuit.barrier()

    for i in range(p, q):
        rotate_state(vqs_generators[i], vqs_params[i], circuit)
    circuit.barrier()

    circuit.x(0)
    controlled_rotate_state(vqs_generators[q], vqs_params[q], circuit)
    circuit.h(0)
    circuit.barrier()
    return measure_ancilla(circuit, shots)

def calculate_v_shots(p, vqs_generators, vqs_params, shots, state, nonlinear):
    n_theta = len(vqs_params)    
    circuit = create_circuit_ancilla(state, nonlinear)

    circuit.h(0)
    circuit.x(0)
    
    for i in range(0, p):
        rotate_state(vqs_generators[i], vqs_params[i], circuit)
    circuit.barrier()

    controlled_rotate_state(vqs_generators[p], vqs_params[p], circuit)
    circuit.barrier()

    for i in range(p, n_theta):
        rotate_state(vqs_generators[i], vqs_params[i], circuit)
    circuit.barrier()

    circuit.x(0)

    coeffs = [0.5, 0.5, -0.5, -0.5, -0.5, -0.5, -1.0]
    measurements = []
    for i in range(len(coeffs)):
        single_h_circuit = copy.deepcopy(circuit)
        controlled_rotate_state(vqs_generators[i], coeffs[i], single_h_circuit)
        single_h_circuit.h(0)
        measurements.append(measure_ancilla(single_h_circuit, shots))
    results = 0
    for i in range(len(coeffs)):
        results += measurements[i] * coeffs[i]
    return results

In [ ]:
def Cost(M,V):
    #f=1/2x^TPx+q^{T}x
    #Gx<=h
    #Ax=b
#     alpha = 0
#     alpha=1e-3
    P=M.T@M+alpha*numpy.eye(len(V))
    q=M.T@V
    thetaDot=solve_qp(P,-q)
    return thetaDot

def McEvolve(vqs_params_init,T,dT,T0,exc_state, way):
    steps=int((T-T0)/dT)
    T_arr=numpy.linspace(T0,T,steps)
    vqs_params=vqs_params_init
    vqs_dot_hist=[]
    vqs_hist=[vqs_params]
    FidelityArr=[]
    GF_exact=[]
    GF_sim=[]
    U = 2
    for i in tqdm(range(len(T_arr))):
        
        #evaluations at time step t_i
        U_T=TimeEvolutionOperator(T_arr[i])
        
        #exact state
        exact_evolved_state=numpy.dot(U_T,exc_state)
        vqs_state=rotated_state(vqs_generators,vqs_hist[-1], exc_state)

        G1=np.exp(1j*(U-rho)/2)*numpy.dot(np.conj(exc_state2), exact_evolved_state).real
        GF_exact.append(G1)
        G2=np.exp(1j*(U-rho)/2)*numpy.dot(np.conj(exc_state2), vqs_state).real
        GF_sim.append(G2)
#         print("Green functions",G1,G2)
   
        FidelityArr.append(numpy.abs(numpy.dot(vqs_state,numpy.conjugate(exact_evolved_state)))**2)
        print("Fidelity",FidelityArr[-1])    
        
        arr = [(j,k) for j in range(len(vqs_params)) for k in range(len(vqs_params)) if j<=k]
        M_mat = numpy.zeros((len(vqs_params),len(vqs_params)))

        #constructing McLachlan
        if way == 'Anirban':
            M_elems = Parallel(n_jobs=-1,verbose=0)(delayed(M)(arr[i][0],arr[i][1],vqs_params,exc_state) for i in range(len(arr)))
            V_vec=numpy.array([V(p,vqs_params,exc_state) for p in range(len(vqs_params))])
        
        # Statevector way
        elif way == 'statevector':
            M_elems = Parallel(n_jobs=-1)(delayed(calculate_m_statevector)(arr[i][0], arr[i][1], vqs_generators, vqs_params, 'state2') for i in range(len(arr)))       
            V_vec = Parallel(n_jobs=-1)(delayed(calculate_v_statevector)(p, vqs_generators, vqs_params, 'state2') for p in range(len(vqs_params)))
        
        # Shots way
        elif way == 'shots':
            shots = 2**15
            M_elems = Parallel(n_jobs=-1)(delayed(calculate_m_shots)(arr[i][0], arr[i][1], vqs_generators, vqs_params, shots, 'state2') for i in range(len(arr)))       
            V_vec = Parallel(n_jobs=-1)(delayed(calculate_v_shots)(p, vqs_generators, vqs_params, shots, 'state2') for p in range(len(vqs_params)))

        for p in range(len(arr)):
            M_mat[arr[p][0]][arr[p][1]] = M_mat[arr[p][1]][arr[p][0]] = M_elems[p]
            
        vqs_params_dot=Cost(M_mat,V_vec)#numpy.linalg.lstsq(M_mat,V_vec,rcond=None)[0]
        vqs_dot_hist.append(vqs_params_dot)
        
        
#         def Error(vqs_params_dot):
#             quant=numpy.sum((M_mat@vqs_params_dot-V_vec)@(M_mat@vqs_params_dot-V_vec).T)
#             print(quant)
#             return quant
#         error=Error(vqs_params_dot)
#         print("Initial Error after least squares-", error)
        
        #Euler
        #vqs_params=vqs_params+vqs_dot_hist[-1]*dT
        #Adams-Bashforth
    
        if i>0:
            vqs_params=vqs_params+1.5*dT*vqs_dot_hist[-1]-0.5*dT*vqs_dot_hist[-2]
        else:
            vqs_params=vqs_params+vqs_dot_hist[-1]*dT 
        vqs_hist.append(vqs_params)
    
    return vqs_hist,FidelityArr,GF_sim,GF_exact    

In [ ]:
# Single optimization
T=5
dT=0.1

nd=2
vqs_generators=['ZIZI','IZIZ','IIXX','IIYY','XXII','YYII','IIII']*nd
vqs_params=numpy.zeros(len(vqs_generators))

# vqs_params_history,FidelityArr,GF_sim,GF_exact=McEvolve(vqs_params,T,dT,0,exc_state, 'statevector')

In [ ]:
# fig, ax = plt.subplots(dpi=160)
# ax.set_title('t=30, dt=0.1, U=2')
# T_arr
# ax.plot(GF_sim, label = 'VQS - statevector', color = 'blue')
# ax.plot(GF_exact, label = 'Exact', color = 'red')
# plt.legend()
# plt.show()

In [ ]:
# # Spectral function plot
# G_sim = GF_sim
# G_exact = GF_exact
# # Number of sample points
# num_samp=len(G_sim)
# # sample spacing
# ImgG_1f = numpy.fft.fft(numpy.real(G_sim))
# ImgG_2f = numpy.fft.fft(numpy.real(G_exact))
# plt.rcParams["figure.figsize"] = (20,10)
# Tf = numpy.linspace(0.0, 1//(2.0*dT), num_samp//2)
# fig, ax = plt.subplots()
# ax.set_xlabel(r'$\omega$',fontsize=20)
# ax.tick_params(labelsize=20)
# ax.set_yscale('log')
# ax.plot(Tf, 2.0/num_samp * numpy.abs(ImgG_1f[:num_samp//2])/numpy.pi,marker='s',color='b',linestyle='',label=r'$Im G_{VHS - statevector}^{1,2}(1,2,\omega)$')
# ax.plot(-Tf, 2.0/num_samp * numpy.abs(ImgG_1f[:num_samp//2])/numpy.pi,marker='s',color='b',linestyle='')
# ax.plot(Tf, 2.0/num_samp * numpy.abs(ImgG_2f[:num_samp//2])/numpy.pi,color='r',linestyle='-',label=r'$Im G_{exact}^{1,2}(1,2,\omega)$')
# ax.plot(-Tf, 2.0/num_samp * numpy.abs(ImgG_2f[:num_samp//2])/numpy.pi,color='r',linestyle='-')
# plt.legend(fontsize=15)
# plt.show()

Find  a circuit rep of U(theta) such that
$U(\theta)|G\rangle \approx e^{-iHT}|G\rangle$ and $U(\theta)|E\rangle \approx e^{-iHT}|E\rangle$.<br>
$U(\theta)|G\rangle \approx e^{-iHT}|G\rangle\to M_{1}\dot{\theta}=V_{1}$, $U(\theta)|G\rangle \approx e^{-iHT}|E\rangle\to M_{2}\dot{\theta}=V_{2}$<br>
Map this to a quadratic optimization problem<br>
$(\dot{\boldsymbol{\theta}}^{T}M_{1}^{T}-V_{1}^{T})(M_{1}\dot{\boldsymbol{\theta}}-V_{1})=\dot{\boldsymbol{\theta}}^{T}M_{1}^{T}M_{1}\dot{\boldsymbol{\theta}}-\dot{\boldsymbol{\theta}}^{T}M_{1}^{T}V_{1}-V^{T}_{1}M_{1}\dot{\boldsymbol{\theta}}+V_{1}^{T}V_{1}\propto \frac{1}{2}\dot{\boldsymbol{\theta}}^{T}M_{1}^{T}M_{1}\dot{\boldsymbol{\theta}}-(M_{1}^{T}V_{1})^{T}\dot{\boldsymbol{\theta}}$<br>
$(\dot{\boldsymbol{\theta}}^{T}M_{1}^{T}-V_{1}^{T})(M_{1}\dot{\boldsymbol{\theta}}-V_{1})+(\dot{\boldsymbol{\theta}}^{T}M_{2}^{T}-V_{2}^{T})(M_{2}\dot{\boldsymbol{\theta}}-V_{2})\propto\frac{1}{2}\dot{\boldsymbol{\theta}}^{T}(M_{1}^{T}M_{1}+M_{2}^{T}M_{2})\dot{\boldsymbol{\theta}}-\left[(M_{1}^{T}V_{1})^{T}+(M_{2}^{T}V_{2})^{T}\right]\dot{\boldsymbol{\theta}}$<br>
Cost Function<br>
$Cost=\frac{1}{2}\dot{\boldsymbol{\theta}}^{T}(M_{1}^{T}M_{1}+M_{2}^{T}M_{2})\dot{\boldsymbol{\theta}}-\left[(M_{1}^{T}V_{1})^{T}+(M_{2}^{T}V_{2})^{T}\right]\dot{\boldsymbol{\theta}}$
<br>
$P=(M_{1}^{T}M_{1}+M_{2}^{T}M_{2})+\alpha$, $\alpha= $Tikhonov Regularization<br>
$q=M^{T}V$<br>
$f=1/2x^TPx+q^{T}x$, $x=\dot{\theta}$

In [ ]:
def seq(start, stop, step):
    n = int(round((stop - start)/float(step)))
    return np.array([np.around(start + step*i,3) for i in range(n+1)])

In [ ]:
def percent_error(a,e):
    return str((np.abs((a-e)/e))*100) + '%'

In [ ]:
def JointCost(M1, V1, M2, V2):
    #f=1/2 {x^T} Px + q^{T}x
    #Gx<=h
    #Ax=b
    alpha = 1e-3
    P = M1.T@M1 + M2.T@M2 + alpha * np.eye(len(V1))
    q = M1.T@V1 + M2.T@V2
#     thetaDot = numpy.linalg.lstsq(P, -q, rcond=None)[0]
    thetaDot = solve_qp(P, -q)
    return thetaDot

def McEvolveJointOptimization(vqs_generators, vqs_params_init, T, dT, T0, state1, state2, method, excitations, nonlinear):
    T_arr = seq(T0, T, dT)
    vqs_params = vqs_params_init
    vqs_dot_hist = []
    vqs_hist = [vqs_params]
    FidelityArr = []
    
    global T1_params
    if not nonlinear['bool']:
        T1_params[0].append(vqs_params_init)
            
    for i in range(len(T_arr)):
        
        if (T_arr[i] != T and not nonlinear['bool']) or (nonlinear['bool']):

            U_T = TimeEvolutionOperator(T_arr[i] - T_arr[0])
            exact_evolved_state1 = U_T@state1 # ground state           
            exact_evolved_state2 = U_T@state2 # excited state
  
            vqs_state1 = rotated_state(vqs_generators,vqs_hist[-1], state1)
            vqs_state2 = rotated_state(vqs_generators,vqs_hist[-1], state2)
  
            FidelityArr.append([numpy.abs(vqs_state1@numpy.conjugate(exact_evolved_state1))**2,numpy.abs(vqs_state2@numpy.conjugate(exact_evolved_state2))**2])
            print("Time: ", np.around(T_arr[i],2), " | Fidelity: ", FidelityArr[-1])    

            if (not nonlinear['bool']) or (nonlinear['bool'] and T_arr[i] != T):
                #constructing McLachlan
                arr = [(j,k) for j in range(len(vqs_params)) for k in range(len(vqs_params)) if j <= k]
                M1 = numpy.zeros((len(vqs_params),len(vqs_params)))
                M2 = numpy.zeros((len(vqs_params),len(vqs_params)))

                # Statevector way
                if method == 'statevector':
                    M_elems1 = Parallel(n_jobs=-1)(delayed(calculate_m_statevector)(arr[q][0], arr[q][1], vqs_generators, vqs_params, 'state_g', excitations, nonlinear) for q in range(len(arr)))       
                    M_elems2 = Parallel(n_jobs=-1)(delayed(calculate_m_statevector)(arr[q][0], arr[q][1], vqs_generators, vqs_params, 'exc_state', excitations, nonlinear) for q in range(len(arr)))       
                    V1 = Parallel(n_jobs=-1)(delayed(calculate_v_statevector)(p, vqs_generators, vqs_params, 'state_g', excitations, nonlinear) for p in range(len(vqs_params)))
                    V2 = Parallel(n_jobs=-1)(delayed(calculate_v_statevector)(p, vqs_generators, vqs_params, 'exc_state', excitations, nonlinear) for p in range(len(vqs_params)))

                # Shots way
                elif method == 'shots':
                    shots = 2**14
                    M_elems1 = Parallel(n_jobs=-1)(delayed(calculate_m_shots)(arr[i][0], arr[i][1], vqs_generators, vqs_params, shots, 'state1', nonlinear) for i in range(len(arr)))       
                    M_elems2 = Parallel(n_jobs=-1)(delayed(calculate_m_shots)(arr[i][0], arr[i][1], vqs_generators, vqs_params, shots, 'state2', nonlinear) for i in range(len(arr)))       
                    V1 = Parallel(n_jobs=-1)(delayed(calculate_v_shots)(p, vqs_generators, vqs_params, shots, 'state1', nonlinear) for p in range(len(vqs_params)))
                    V2 = Parallel(n_jobs=-1)(delayed(calculate_v_shots)(p, vqs_generators, vqs_params, shots, 'state2', nonlinear) for p in range(len(vqs_params)))

                for p in range(len(arr)):
                    M1[arr[p][0]][arr[p][1]] = M1[arr[p][1]][arr[p][0]] = M_elems1[p]
                    M2[arr[p][0]][arr[p][1]] = M2[arr[p][1]][arr[p][0]] = M_elems2[p]

                vqs_params_dot = JointCost(M1, V1, M2, V2)        
                vqs_dot_hist.append(vqs_params_dot)

                if i == 0:
                    vqs_params = vqs_params + vqs_dot_hist[-1]*dT 
                else:
                    vqs_params = vqs_params + (3/2)*dT*vqs_dot_hist[-1]-(1/2)*dT*vqs_dot_hist[-2]

                vqs_hist.append(vqs_params)
                if not nonlinear['bool']:
                    T1_params[0].append(vqs_params)
                else:
                    T1_params[1].append(vqs_params)

                np.set_printoptions(precision=4, suppress=True)
                # add third excitation if we are optimizing nonlinearly
                if nonlinear['bool']:
                    global single_percent_error
                    evolved_params2 = vqs_hist[-1]
                    vqs_state_g = rotated_state(vqs_generators, evolved_params2, state1)
                    vqs_exc_state = rotated_state(vqs_generators, evolved_params2, state2)  
                    
                    exact_state_g = exact_states(T_arr[0],T_arr[i+1] - T_arr[0])[0]
                    exact_exc_state = exact_states(T_arr[0],T_arr[i+1] - T_arr[0])[1]

                    vqs_G_term = vqs_state_g.conj()@vqs_exc_state
                    exact_G_term = exact_state_g.conj()@exact_exc_state
                    print()
                    print('Ground state inner product: ', vqs_state_g.conj()@exact_state_g)
                    print('Excited state inner product: ', vqs_exc_state.conj()@exact_exc_state)
                    
                    vqs_G_term = 0 #G_term sums over the P3 excitations
                    exact_G_term = 0
                    
                    for j in range(len(P3_arr)):
                        new_exc_labels = [P3_arr[j]]
                        new_exc_params = numpy.array([numpy.pi/2])
                        vqs_exc_state_p3 = rotated_state(new_exc_labels, new_exc_params, vqs_exc_state)
                        vqs_G_term += (coeffs[j] * vqs_state_g.conj()@vqs_exc_state_p3)

                        exact_exc_state_p3 = rotated_state(new_exc_labels, new_exc_params, exact_exc_state)
                        exact_G_term += (coeffs[j] * exact_state_g.conj()@exact_exc_state_p3)
                        
                    
                    print('G percent error for ', T_arr[0], np.around(T_arr[i+1] - T_arr[0],2), ' : ', percent_error(vqs_G_term*4, exact_G_term*4))
                    single_percent_error.append(np.abs((vqs_G_term*4-exact_G_term*4)/(exact_G_term*4))*100)
    #                 global G
    #                 G.append(G_term * 4) #there are 4 G_to append, but they are the same so we multiply by 4

                
    return vqs_hist,FidelityArr   

# Non-linear Joint Optimization

In [ ]:
def NLJointOptimization(T0, T1, T2, alpha, depth, shots, excitations):
    
    #  first excitation 
    exc_labels = excitations[0]
    exc_params = numpy.array([-numpy.pi/2,numpy.pi/2])
    exc_state = rotated_state(exc_labels,exc_params,state_g)

    # first evolution
    vqs_generators = ['ZIZI','IZIZ','IIXX','IIYY','XXII','YYII','IIII'] * depth
    vqs_params = numpy.zeros(len(vqs_generators))
    nonlinear = {'bool' : False, 'generators': vqs_generators, 'params': vqs_params}
    vqs_hist, FidelityArr = McEvolveJointOptimization(vqs_generators, vqs_params, T1, dT, T0, state_g, exc_state, 'statevector', excitations, nonlinear)
    evolved_params = vqs_hist[-1]
    vqs_state_g = rotated_state(vqs_generators,evolved_params,state_g)
    vqs_exc_state = rotated_state(vqs_generators,evolved_params,exc_state)

    # seocnd excitation (which only applies to the excited state)
    exc_labels2 = excitations[1]
    exc_params2 = numpy.array([-numpy.pi/2,numpy.pi/2])
    vqs_exc_state = rotated_state(exc_labels2, exc_params2, vqs_exc_state)
    
    # second evolution
    nonlinear = {'bool' : True, 'generators': vqs_generators, 'params': evolved_params}
    vqs_hist2, FidelityArr2 = McEvolveJointOptimization(vqs_generators, vqs_params, T2, dT, T1, vqs_state_g, vqs_exc_state, 'statevector', excitations, nonlinear)
    evolved_params2 = vqs_hist2[-1]
    vqs_state_g = rotated_state(vqs_generators, evolved_params2, vqs_state_g)
    vqs_exc_state = rotated_state(vqs_generators, evolved_params2, vqs_exc_state)  
    
    combined_fidelity = np.array(FidelityArr + FidelityArr2)

    return combined_fidelity

# Begin simulations

In [ ]:
excitation_pair = (['IIII', 'IXII'], ['IIII', 'IIIX'])
P3_arr = ['XZXZ', 'YZXZ', 'XZYZ', 'YZYZ']
coeffs = [1, 1j, 1j, -1]
alpha = 0.001
depth = 4
vqs_generators = ['ZIZI','IZIZ','IIXX','IIYY','XXII','YYII','IIII'] * depth

G = []
T1 = 5
tau = 5
fidelities = []
percent_errors = []
depth = 4
dT_list = [1/20, 1/30, 1/50]
for dT in dT_list:
    print(dT)
    single_percent_error = []
    T1_params = [[] for i in range(2)]
    combined_fidelity = NLJointOptimization(0, T1, T1+tau, alpha, depth, None, excitation_pair)
    percent_errors.append(np.mean(single_percent_error))
    fidelities.append(combined_fidelity)

In [ ]:
depth_list = [str(depth) for depth in depth_list]

In [ ]:
dT_list = [1/20, 1/30, 1/50]
dT_list = [np.around(dT,3) for dT in dT_list]
dT_list = [str(dT) for dT in dT_list]

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_title("T1=5, T2=5, depth=4")
ax.set_xlabel('dT')
ax.set_ylabel('Average Percent Error')

ax.bar(dT_list, percent_errors)
plt.show()

In [ ]:
# Excitation pair plot
colors = plt.cm.cividis(np.linspace(0, 1, len(percent_errors)))
colors = np.flip(colors, axis=0)
fig, ax = plt.subplots(dpi=500)
ax.set_xlabel('Time')
ax.set_ylabel('Percent Error')
ax.set_title("T1=1, T2=2, depth=4")
ax.bar(str(dT_list), percent_errors)

for i in range(len(percent_errors)):
    ax.plot(dT_list, percent_errors[i])
# plt.legend(title="Excitation pair")
plt.show()

In [ ]:
G2 = []
for tau in seq(0.1,T2-T1,0.1):
    G2.append(GF(0.5,tau))

if len(G) != len(G2):
    del G[-1]

for i in range(len(G)):
    print(G2[i]-G[i])

In [ ]:
# Excitation pair plot
colors = plt.cm.cividis(np.linspace(0, 1, len(fidelities)))
colors = np.flip(colors, axis=0)
fig, ax = plt.subplots(dpi=500)
ax.set_xlabel('Time')
ax.set_ylabel('G_exact - G_statevector')
ax.set_title("T1=0.5, T2=1.0, dT=0.1, depth=4")
# ax.set_ylim([0.9998,1])

T_arr = seq(0.6,1.0,0.1)
for i in range(len(fidelities)):
    ax.plot(T_arr, np.array(G2)-np.array(G))
# plt.legend(title="Excitation pair")
plt.show()

In [ ]:
# Excitation pair plot
colors = plt.cm.cividis(np.linspace(0, 1, len(fidelities)))
colors = np.flip(colors, axis=0)
fig, ax = plt.subplots(dpi=500)
ax.set_xlabel('Time')
ax.set_ylabel('G_exact - G_statevector')
ax.set_title("T1=5, T2=10, dT=0.1, depth=4")
# ax.set_ylim([0.9998,1])

T_arr = seq(5,9.9,0.1)
for i in range(len(fidelities)):
    ax.plot(T_arr, np.array(G2)-np.array(G))
# plt.legend(title="Excitation pair")
plt.show()

In [ ]:
# Excitation pair plot
colors = plt.cm.cividis(np.linspace(0, 1, len(fidelities)))
colors = np.flip(colors, axis=0)
fig, ax = plt.subplots(dpi=500)
ax.set_xlabel('Time')
ax.set_ylabel('Excited state fidelity')
ax.set_title("T1=1, T2=2, dT=0.1, depth=4")
ax.set_ylim([0.9998,1])

T_arr = seq(0,T2,dT)
for i in range(len(fidelities)):
    ax.plot(T_arr, np.mean(fidelities[i],axis=1), label=excitation_pairs[i], color=colors[i])
plt.legend(title="Excitation pair")
plt.show()

In [ ]:
# grid params
excitation_pair = (['IIII', 'IXII'], ['IIII', 'IIIX'])
coeffs = [1, 1j, -1, 1j]
P3_arr = ['XYII','XXII','YYII','YXII']
alpha = 0.001
depth = 2
vqs_generators = ['ZIZI','IZIZ','IIXX','IIYY','XXII','YYII','IIII'] * depth

grid_params = []
T1_list = seq(0.1,5,0.1)
tau = 5
for T1 in T1_list:
    T1_params = [[] for i in range(2)]
    T2 = T1 + tau
    dT = 0.1
    combined_fidelity = NLJointOptimization(0, T1, T2, alpha, depth, None, excitation_pair)
    grid_params.append(T1_params)
np.save('grid_params_5_10', grid_params)

# Begin calculations / plots

In [ ]:
def GF(T1, tau):

    P3_arr=['XZXZ', 'YZXZ', 'XZYZ', 'YZYZ']
    coeff=[1, 1j, 1j, -1]
    exc_labels_arr = [
                (['IIII', 'IXII'], ['IIII', 'IIIX']), 
#                 (['IIII', 'IYII'], ['IIII', 'IIIY']),
#                 (['IIII', 'IXII'], ['IIII', 'IIIY']),
#                 (['IIII', 'IYII'], ['IIII', 'IIIX'])
              ]
    G = 0
    
    for i in range(len(exc_labels_arr)):
        
        excitations = exc_labels_arr[i]
        # first excitation
        exc_labels = excitations[0]
        exc_params = numpy.array([-numpy.pi/2,numpy.pi/2])
        exc_state = rotated_state(exc_labels, exc_params, state_g)
        # first evolution
        U_T1 = TimeEvolutionOperator(T1)
        exact_evolved_state = U_T1@exc_state #evolved after T1
        
        # second excitation
        exc_labels1 = excitations[1]
        exc_params1 = numpy.array([-numpy.pi/2,numpy.pi/2])
        state1 = rotated_state(exc_labels1, exc_params1, exact_evolved_state)

        first_ground = U_T1@state_g

        # second evolution
        U_tau = TimeEvolutionOperator(tau)
        state = U_tau@state1
                
        G_term = 0 
        for j in range(len(P3_arr)):
            # final excitation
            exc_labels2 = [P3_arr[j]]
            exc_params2 = numpy.array([numpy.pi/2])
            evolved_state = rotated_state(exc_labels2, exc_params2, state)
            
            # evolution of ground state
            U_T2 = TimeEvolutionOperator(T1+tau)
            state2 = U_T2@state_g 
            
            multiplier = 1
            print(multiplier * coeff[j] * numpy.conjugate(state2)@evolved_state)
            G_term += (multiplier * coeff[j] * numpy.conjugate(state2)@evolved_state)        
        G = G + G_term
        print()
    return G * 4

In [ ]:
def exact_states(T1, tau):
    
    excitations = [['IIII', 'IXII'], ['IIII', 'IIIX']] 
    # first excitation
    exc_labels = excitations[0]
    exc_params = numpy.array([-numpy.pi/2,numpy.pi/2])
    global state_g
    exc_state = rotated_state(exc_labels, exc_params, state_g)
    # first evolution
    U_T1 = TimeEvolutionOperator(T1)
    exact_evolved_state = U_T1@exc_state #evolved after T1

    # second excitation
    exc_labels1 = excitations[1]
    exc_params1 = numpy.array([-numpy.pi/2,numpy.pi/2])
    state1 = rotated_state(exc_labels1, exc_params1, exact_evolved_state)

    U_tau = TimeEvolutionOperator(tau)
    U_T2 = TimeEvolutionOperator(T1+tau)

    return [U_T2@state_g, U_tau@state1]